This stem mapping technique assumes that there are enough different scanning positions to fully capture each tree trunk. Therefore it's helpful to have some way to describe if a tree is occluded, and if so, how much.

In [ ]:
# load the necessary libraries

import matplotlib.pyplot as plt
import numpy as np

from math import ceil

from PIL import Image

from skimage.color import rgb2gray
from skimage.draw import circle_perimeter
from skimage.feature import canny
from skimage.morphology import binary_closing, disk
from skimage.transform import rotate, rescale

In [ ]:
tree_trunk = Image.open('data/tree_trunk.jpg')
tree_trunk = np.array(tree_trunk)

plt.imshow(tree_trunk)

We can see that this tree trunk is mostly circular, and prove it by overlaying a circle with *radius* 28 at (34,35).

In [ ]:
copy = tree_trunk.copy()
sub_y, sub_x = circle_perimeter(34, 35, 28)
copy[sub_y, sub_x] = (255)

plt.imshow(copy)

Note the occluded section in the lower right at (45, 60).

In [ ]:
occluded = tree_trunk.copy()
sub_y, sub_x = circle_perimeter(60, 45, 7)
occluded[sub_y, sub_x] = (255)

plt.imshow(occluded)

I've worked with this data long enough to know that the circled section is occlusion, while the rest of the tree trunk has been captured by the scanner, albeit with low intensity at several places. Therefore I'm going to do some transformations on the image to make the difference between low coverage (low intensity) and occlusion (no or very low intensity) more clear.

First up is a binary threshold. I'm going to use the upper quartile as the threshold in this case.

In [ ]:
max_trunk = tree_trunk.copy()

intensities = max_trunk[max_trunk != 0]

threshold = np.quantile(intensities, 0.75)

max_trunk = np.where(max_trunk > threshold, 255, 0)

plt.imshow(max_trunk)

Note the stripes along the image. These are due to the high resolution of the scanner and the texture of the tree trunk rather than the number of scan positions. They definitely are NOT gaps in coverage, so let's fix them with a morphological close. This will *close* the small gaps between pixels.

In [ ]:
closed_max = binary_closing(max_trunk, disk(2))
plt.imshow(closed_max)

Now that we have highlighted the areas where we have good coverage, an edge deetction followed by another close will make the true occlusion stand out.

In [ ]:
enhanced_trunk = canny(closed_max, sigma=1)

plt.imshow(enhanced_trunk)

In [ ]:
closed_edge = binary_closing(enhanced_trunk, disk(2))

plt.imshow(closed_edge)

Now it's much more obvious that there was something close to tree, perhaps a branch, that obscured a small part of the trunk. How can we measure that?

Let's see how much of our fitted circle from earlier intersects with our closed edge image.

In [ ]:
obs = np.empty_like(closed_edge)
obs[:,:] = closed_edge
pred = np.zeros(closed_edge.shape)

sub_y, sub_x = circle_perimeter(34, 35, 28)
pred[sub_y, sub_x] = (255)

intersection = pred*obs

plt.imshow(intersection)

We can see that the majority of the fitted circle is captured. We can calculate a ratio by using pixel counts.

In [ ]:
pred_count = np.count_nonzero(pred)
obs_count = np.count_nonzero(intersection)
print(pred_count)
print(obs_count)

ratio = obs_count/pred_count
print(ratio)

We can say that this tree trunk has ~90% coverage, which is pretty good! In addition to the occlusion, there was a small fraction of the trunk that didn't meet the circular assumption in the upper left. Now let's look at a different tree and see how it performs. 

In [ ]:
occluded_trunk = Image.open('data/occluded_trunk.jpg')
occluded_trunk = np.array(occluded_trunk)

plt.imshow(occluded_trunk)

The tree is centered on (24, 22) with a radius of 11.

In [ ]:
copy = occluded_trunk.copy()
sub_y, sub_x = circle_perimeter(22, 24, 11)
copy[sub_y, sub_x] = (255)

plt.imshow(copy)

There is a lot more occlusion here, so let's what our new metrics will show. First, let's apply the transforms from earlier.

In [ ]:
max_occlusion = occluded_trunk.copy()

intensities = max_occlusion[max_occlusion != 0]

threshold = np.quantile(intensities, 0.75)

max_occlusion = np.where(max_occlusion > threshold, 255, 0)

plt.imshow(max_occlusion)

In [ ]:
closed_max = binary_closing(max_occlusion, disk(2))
plt.imshow(closed_max)

In [ ]:
enhanced_occlusion = canny(closed_max, sigma=1)

plt.imshow(enhanced_occlusion)

In [ ]:
closed_edge = binary_closing(enhanced_occlusion, disk(2))

plt.imshow(closed_edge)

While it's clear that *something* is there, it's also pretty obvious why a circle doesn't fit very well here.

In [ ]:
obs = np.empty_like(closed_edge)
obs[:,:] = closed_edge
pred = np.zeros(closed_edge.shape)

sub_y, sub_x = circle_perimeter(22, 24, 11)
pred[sub_y, sub_x] = (255)

intersection = pred*obs

plt.imshow(intersection)

In [ ]:
pred_count = np.count_nonzero(pred)
obs_count = np.count_nonzero(intersection)
print(pred_count)
print(obs_count)

ratio = obs_count/pred_count
print(ratio)

Now we have a description that lets us compare tree trunks based on how well they fit a circular assumption. As a final step, we can extend this logic to look at slices of the circle and examine how well each portion is formed. 

In [ ]:
# let's define some functions

def pizzaSlice(np_img, theta):
    nrows, ncols = np_img.shape
    out = np_img.copy()
    half = ceil(nrows/2)
    for i in range(0, half):
        out[:,i] = 0
    out = rotate(out, theta)
    for i in range(0, half):
        out[:,-1*(i+1)] = 0
    out = rotate(out, -1*theta)

    return out

def radialSlices(np_img, theta):
    num_slices = ceil(360/theta)
    slice_out = []
    for i in range(0, num_slices):
        img = rotate(np_img, i*theta)
        out = pizzaSlice(img, theta)
        slice_out.append(out)
    return slice_out

I like using this clock as an extra example, since it's easier to see what's going on.

In [ ]:
clock = Image.open('data/clock.jpeg')
clock = np.array(clock)
clock = rgb2gray(clock)

plt.imshow(clock)

In [ ]:
clock_slice = pizzaSlice(clock, 90)
plt.imshow(clock_slice)

In [ ]:
theta = 90

clocks = radialSlices(clock, theta)
for i in clocks:
    plt.figure()
    plt.imshow(i)

Now let's apply that to the occluded tree trunk from earlier.

In [ ]:
theta = 90

occluded_slices = radialSlices(closed_edge, theta)
for i in occluded_slices:
    plt.figure()
    plt.imshow(i)

In [ ]:
pred = np.zeros(closed_edge.shape)

sub_y, sub_x = circle_perimeter(22, 24, 11)
pred[sub_y, sub_x] = (255)

inters = []
ratios = []

pred_count = np.count_nonzero(pred)

for i in occluded_slices:
    obs = np.empty_like(i)
    obs[:,:] = i
    intersection = pred*obs
    inters.append(intersection)
    obs_count = np.count_nonzero(intersection)
    ratios.append(obs_count/pred_count)

for i in inters:
    plt.figure()
    plt.imshow(i)

We can also look at the ratios for each section.

In [ ]:
for i in ratios:
    print(i)